### Iterate through PH domains in the 2 

In [2]:
import pandas as pd
from pathlib import Path
import IMP
import IMP.atom
import IMP.rmf
import RMF
import random
import numpy as np
import matplotlib.pyplot as plt


In [3]:
gsms_dir = Path(Path.home(), "Documents/mtorc2/tmp/1000R")
gsms_file = Path(gsms_dir, "A.rmf3")
fh_clust = RMF.open_rmf_file_read_only(str(gsms_file))
m_clust = IMP.Model()
h_clust = IMP.rmf.create_hierarchies(fh_clust, m_clust)[0]


In [23]:
df_data = pd.DataFrame(index=range(1000),columns=["d_MSIN1", "d_AKT1"])

for i in range(1000):
    IMP.rmf.load_frame(fh_clust, RMF.FrameID(i))

    for mol in ["MSIN1", "AKT1"]:
        if mol == "MSIN1":
            lower_bound, upper_bound = 383,522
        elif mol == "AKT1":
            lower_bound, upper_bound = 1,143

        sel = IMP.atom.Selection(h_clust, resolution=1, molecule=mol, residue_indexes=list(range(lower_bound,upper_bound)), copy_index=0)

        pids = [pid for pid in sel.get_selected_particle_indexes()]

        xyzs = [IMP.core.XYZ(m_clust, pid) for pid in pids]

        cum_dist = 0
        for xyz in xyzs:
            z_coord = xyz.get_coordinates()[2]
            cum_dist = cum_dist + (232.481 - z_coord)

        avg_dist = cum_dist / len(xyzs)
        df_data.iloc[i]["d_{}".format(mol)] = avg_dist

# del fh_clust


In [24]:
df_data["d_AKT1"] = pd.to_numeric(df_data["d_AKT1"])
df_data["d_MSIN1"] = pd.to_numeric(df_data["d_MSIN1"])

df_data["sum"] = df_data["d_AKT1"] + df_data["d_MSIN1"]
df_data.head()

,d_MSIN1,d_AKT1,sum
0,64.322228,41.224826,105.547054
1,57.217019,46.900483,104.117502
2,44.741249,47.071309,91.812558
3,98.222770,57.173985,155.396755
4,51.796075,64.440933,116.237008


In [27]:
df_data.nsmallest(25, "d_AKT1")

,d_MSIN1,d_AKT1,sum
775,7.029784,26.223675,33.253459
432,106.482649,26.796458,133.279107
102,81.255173,27.426961,108.682135
868,-5.997882,30.089767,24.091885
745,65.274001,30.213163,95.487164
522,90.348881,30.598638,120.947519
222,83.356068,30.757981,114.114049
438,-14.060986,30.991132,16.930146
872,71.578619,31.136644,102.715262
340,36.834020,31.811026,68.645046


In [30]:
pdb_dir = Path(Path.home(), "Documents/mtorc2/tmp/membrane")
for frame_id in list(pd.DataFrame(df_data).nsmallest(25, "d_AKT1").index):
    print(frame_id)
    IMP.rmf.load_frame(fh_clust, RMF.FrameID(frame_id))

    sel = IMP.atom.Selection(h_clust, resolution=1, molecules=["MSIN1", "AKT1"])

    IMP.atom.write_pdb(sel, str(Path(pdb_dir, str(frame_id)+".pdb")))



775
432
102
868
745
522
222
438
872
340
681
610
822
679
812
162
434
734
758
919
574
814
182
25
336
